In [5]:
import requests
import json
import prettytable
import datetime

# PCU22112222112241
# CUUR0000SA0L1E

# Get the current year and previous year
current_year = datetime.date.today().year
previous_year = current_year - 1

# Insert the years into the URL
data = json.dumps({"seriesid": ['PCU22112222112241','CUUR0000SA0L1E'], "startyear": str(previous_year), "endyear": str(current_year)})
headers = {'Content-type': 'application/json'}
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
print(p.status_code)
json_data = json.loads(p.text)

ConnectionError: HTTPSConnectionPool(host='api.bls.gov', port=443): Max retries exceeded with url: /publicAPI/v2/timeseries/data/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0xffff889bb7c0>: Failed to resolve 'api.bls.gov' ([Errno -2] Name or service not known)"))

In [6]:
for series in json_data['Results']['series']:
    series_name = series['seriesID']
    print(f"Series: {series_name}")

    x = prettytable.PrettyTable(["series id", "year", "period", "value", "footnotes"])
    seriesId = series['seriesID']
    data = series['data']

    # Sort data by year and period
    data.sort(key=lambda x: (int(x['year']), x['period']))

    # Get the latest and previous months
    latest_month = data[-1]
    previous_month = data[-2]

    latest_value = float(latest_month['value'])
    previous_value = float(previous_month['value'])
    change = latest_value - previous_value
    percentage_change = (change / previous_value) * 100

    # Check if there's enough data to calculate 12-month change
    if len(data) < 13:
        print("Not enough data to calculate 12-month change")
    else:
        # Get the same month 12 months ago
        same_month_12_months_ago = next((item for item in data if int(item['year']) == int(latest_month['year']) - 1 and item['period'] == latest_month['period']), None)
        if same_month_12_months_ago:
            same_month_12_months_ago_value = float(same_month_12_months_ago['value'])
            twelve_month_change = latest_value - same_month_12_months_ago_value
            twelve_month_percentage_change = (twelve_month_change / same_month_12_months_ago_value) * 100
            print(f"12-month change: {twelve_month_change:.2f} ({twelve_month_percentage_change:.2f}%)")
        else:
            print("Not enough data to calculate 12-month change")

    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes = ""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId, year, period, value, footnotes[0:-1]])

    output = open(seriesId + '.txt', 'w')
    output.write(x.get_string())
    output.close()

    print(f"Latest month: {latest_month['period']} - Value: {latest_value}")
    print(f"Previous month: {previous_month['period']} - Value: {previous_value}")
    print(f"Change: {change:.2f} ({percentage_change:.2f}%)")
    print()  # Empty line for better readability

Series: PCU22112222112241
12-month change: 5.47 (2.37%)
Latest month: M07 - Value: 236.071
Previous month: M06 - Value: 237.821
Change: -1.75 (-0.74%)

Series: CUUR0000SA0L1E
12-month change: 9.81 (3.17%)
Latest month: M07 - Value: 319.214
Previous month: M06 - Value: 319.003
Change: 0.21 (0.07%)

